In [17]:
%pip install torch torchvision torchaudio transformers datasets nltk numpy pandas



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import nltk
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict



In [19]:
# Load dataset
with open("holbrook-missp.dat", "r") as file:
    raw_data = file.readlines()

# Process data
corrections = {}
current_correct_word = None

for line in raw_data:
    word = line.strip()
    if word.startswith("$"):  # Correct words start with $
        current_correct_word = word[1:]
        corrections[current_correct_word] = []
    else:
        if current_correct_word:
            corrections[current_correct_word].append(word)

# Convert into a DataFrame
df = pd.DataFrame([(k, v) for k, values in corrections.items() for v in values],
                  columns=["Correct_Word", "Misspelled_Word"])

# Split into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print("Training samples:", len(train_df))
print("Testing samples:", len(test_df))
df.head()


Training samples: 1432
Testing samples: 359


,Correct_Word,Misspelled_Word
0,?,Consulatoin 1
1,?,Doc_chm 1
2,?,Half-hose 1
3,?,Hurbured 1
4,?,anetes 1


In [20]:
'''# Function to calculate Levenshtein Distance
def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]

# Function to find the closest correct word
def find_closest_word(misspelled_word, correct_words):
    distances = [(word, levenshtein_distance(misspelled_word, word)) for word in correct_words]
    closest_word = min(distances, key=lambda x: x[1])[0]
    return closest_word

# Evaluate the model on the test set
correct_words = train_df["Correct_Word"].unique()
test_df["Predicted_Correct_Word"] = test_df["Misspelled_Word"].apply(lambda x: find_closest_word(x, correct_words))

# Calculate accuracy
accuracy = np.mean(test_df["Correct_Word"] == test_df["Predicted_Correct_Word"])
print("Accuracy:", accuracy)

test_df.head()'''

'# Function to calculate Levenshtein Distance\ndef levenshtein_distance(s1, s2):\n    if len(s1) < len(s2):\n        return levenshtein_distance(s2, s1)\n\n    if len(s2) == 0:\n        return len(s1)\n\n    previous_row = range(len(s2) + 1)\n    for i, c1 in enumerate(s1):\n        current_row = [i + 1]\n        for j, c2 in enumerate(s2):\n            insertions = previous_row[j + 1] + 1\n            deletions = current_row[j] + 1\n            substitutions = previous_row[j] + (c1 != c2)\n            current_row.append(min(insertions, deletions, substitutions))\n        previous_row = current_row\n\n    return previous_row[-1]\n\n# Function to find the closest correct word\ndef find_closest_word(misspelled_word, correct_words):\n    distances = [(word, levenshtein_distance(misspelled_word, word)) for word in correct_words]\n    closest_word = min(distances, key=lambda x: x[1])[0]\n    return closest_word\n\n# Evaluate the model on the test set\ncorrect_words = train_df["Correct_Word"

In [21]:
'''# Build n-gram frequency models
def build_ngram_model(words, n=2):
    ngrams = [tuple(words[i:i+n]) for i in range(len(words)-n+1)]
    ngram_freq = Counter(ngrams)
    return ngram_freq

correct_words = train_df["Correct_Word"].tolist()
unigram_freq = Counter(correct_words)
bigram_freq = build_ngram_model(correct_words, n=2)

# Function to calculate word likelihood
def word_likelihood(word, unigram_freq, bigram_freq):
    likelihood = unigram_freq[word] / sum(unigram_freq.values())
    for i in range(len(word)-1):
        bigram = (word[i], word[i+1])
        likelihood *= (bigram_freq[bigram] + 1) / (unigram_freq[word[i]] + len(unigram_freq))
    return likelihood

# Function to find the most likely correct word
def find_most_likely_word(misspelled_word, correct_words, unigram_freq, bigram_freq):
    likelihoods = [(word, word_likelihood(word, unigram_freq, bigram_freq)) for word in correct_words]
    most_likely_word = max(likelihoods, key=lambda x: x[1])[0]
    return most_likely_word

# Evaluate the model on the test set
test_df["Predicted_Correct_Word"] = test_df["Misspelled_Word"].apply(lambda x: find_most_likely_word(x, correct_words, unigram_freq, bigram_freq))

# Calculate accuracy
accuracy = np.mean(test_df["Correct_Word"] == test_df["Predicted_Correct_Word"])
print("Accuracy:", accuracy)

test_df.head()'''

'# Build n-gram frequency models\ndef build_ngram_model(words, n=2):\n    ngrams = [tuple(words[i:i+n]) for i in range(len(words)-n+1)]\n    ngram_freq = Counter(ngrams)\n    return ngram_freq\n\ncorrect_words = train_df["Correct_Word"].tolist()\nunigram_freq = Counter(correct_words)\nbigram_freq = build_ngram_model(correct_words, n=2)\n\n# Function to calculate word likelihood\ndef word_likelihood(word, unigram_freq, bigram_freq):\n    likelihood = unigram_freq[word] / sum(unigram_freq.values())\n    for i in range(len(word)-1):\n        bigram = (word[i], word[i+1])\n        likelihood *= (bigram_freq[bigram] + 1) / (unigram_freq[word[i]] + len(unigram_freq))\n    return likelihood\n\n# Function to find the most likely correct word\ndef find_most_likely_word(misspelled_word, correct_words, unigram_freq, bigram_freq):\n    likelihoods = [(word, word_likelihood(word, unigram_freq, bigram_freq)) for word in correct_words]\n    most_likely_word = max(likelihoods, key=lambda x: x[1])[0]\n

In [22]:
'''# Build a model of common spelling mistakes
def build_error_model(df):
    error_model = defaultdict(Counter)
    for _, row in df.iterrows():
        correct_word = row["Correct_Word"]
        misspelled_word = row["Misspelled_Word"]
        for i in range(min(len(correct_word), len(misspelled_word))):
            if correct_word[i] != misspelled_word[i]:
                error_model[correct_word[i]][misspelled_word[i]] += 1
    return error_model

error_model = build_error_model(train_df)

# Function to calculate word probability given the noisy channel model
def word_probability(word, correct_word, error_model):
    probability = 1.0
    for i in range(min(len(word), len(correct_word))):
        if word[i] != correct_word[i]:
            denominator = sum(error_model[correct_word[i]].values()) + len(error_model[correct_word[i]])
            if denominator == 0:
                denominator = 1  # Avoid dividing by zero
            probability *= (error_model[correct_word[i]][word[i]] + 1) / denominator
    return probability

# Function to find the most likely correct word
def find_most_likely_word(misspelled_word, correct_words, error_model):
    probabilities = [(word, word_probability(misspelled_word, word, error_model)) for word in correct_words]
    most_likely_word = max(probabilities, key=lambda x: x[1])[0]
    return most_likely_word

# Evaluate the model on the test set
correct_words = train_df["Correct_Word"].unique()
test_df["Predicted_Correct_Word"] = test_df["Misspelled_Word"].apply(lambda x: find_most_likely_word(x, correct_words, error_model))

# Calculate accuracy
accuracy = np.mean(test_df["Correct_Word"] == test_df["Predicted_Correct_Word"])
print("Accuracy:", accuracy)

test_df.head()'''

'# Build a model of common spelling mistakes\ndef build_error_model(df):\n    error_model = defaultdict(Counter)\n    for _, row in df.iterrows():\n        correct_word = row["Correct_Word"]\n        misspelled_word = row["Misspelled_Word"]\n        for i in range(min(len(correct_word), len(misspelled_word))):\n            if correct_word[i] != misspelled_word[i]:\n                error_model[correct_word[i]][misspelled_word[i]] += 1\n    return error_model\n\nerror_model = build_error_model(train_df)\n\n# Function to calculate word probability given the noisy channel model\ndef word_probability(word, correct_word, error_model):\n    probability = 1.0\n    for i in range(min(len(word), len(correct_word))):\n        if word[i] != correct_word[i]:\n            denominator = sum(error_model[correct_word[i]].values()) + len(error_model[correct_word[i]])\n            if denominator == 0:\n                denominator = 1  # Avoid dividing by zero\n            probability *= (error_model[corr

In [23]:
class SpellingDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.vocab = set("".join(data["Misspelled_Word"]) + "".join(data["Correct_Word"]))
        self.char2idx = {ch: i+1 for i, ch in enumerate(self.vocab)}  # +1 to reserve 0 for padding
        self.idx2char = {i: ch for ch, i in self.char2idx.items()}
        self.max_len = max(data["Misspelled_Word"].apply(len))  # Max word length
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        misspelled = [self.char2idx[ch] for ch in row["Misspelled_Word"]]
        correct = [self.char2idx[ch] for ch in row["Correct_Word"]]
        
        # Pad sequences
        misspelled += [0] * (self.max_len - len(misspelled))
        correct += [0] * (self.max_len - len(correct))
        
        return torch.tensor(misspelled), torch.tensor(correct)

# Load dataset
train_dataset = SpellingDataset(train_df)
test_dataset = SpellingDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [24]:
class Seq2SeqModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=32, hidden_dim=64):
        super(Seq2SeqModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size + 1, embedding_dim)  # +1 for padding
        self.encoder = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.decoder = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size + 1)

    def forward(self, x):
        embedded = self.embedding(x)
        _, (hidden, _) = self.encoder(embedded)
        out, _ = self.decoder(embedded, (hidden, torch.zeros_like(hidden)))
        return self.fc(out)

# Initialize model
vocab_size = len(train_dataset.vocab)
model = Seq2SeqModel(vocab_size)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [25]:
def collate_fn(batch):
    misspelled, correct = zip(*batch)
    misspelled = pad_sequence(misspelled, batch_first=True, padding_value=0)
    correct = pad_sequence(correct, batch_first=True, padding_value=0)
    return misspelled, correct

batch_size = 32  # Define batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn)

In [26]:
for misspelled, correct in train_loader:
    optimizer.zero_grad()
    
    outputs = model(misspelled)  # Forward pass
    
    # Debugging shape mismatch issue
    print(f"Output shape: {outputs.shape}")  # Expected: (batch_size, seq_len, vocab_size)
    print(f"Target shape: {correct.shape}")  # Expected: (batch_size, seq_len)

    batch_size, seq_len, vocab_size = outputs.shape  # Unpack output dimensions

    # Ensure the correct tensor is the same shape as the output tensor
    correct = correct[:, :seq_len]

    # Reshape tensors correctly for loss computation
    loss = criterion(
        outputs.reshape(batch_size * seq_len, vocab_size),  # Flatten output for cross-entropy
        correct.reshape(-1)  # Flatten target labels
    )

    loss.backward()
    optimizer.step()
    
    print(f"Loss: {loss.item()}")  # Monitor loss


Output shape: torch.Size([32, 15, 64])
Target shape: torch.Size([32, 15])
Loss: 4.219175338745117
Output shape: torch.Size([32, 15, 64])
Target shape: torch.Size([32, 15])
Loss: 4.183636665344238
Output shape: torch.Size([32, 15, 64])
Target shape: torch.Size([32, 15])
Loss: 4.135247230529785
Output shape: torch.Size([32, 15, 64])
Target shape: torch.Size([32, 15])
Loss: 4.089835166931152
Output shape: torch.Size([32, 15, 64])
Target shape: torch.Size([32, 15])
Loss: 4.0490827560424805
Output shape: torch.Size([32, 15, 64])
Target shape: torch.Size([32, 15])
Loss: 4.003973960876465
Output shape: torch.Size([32, 15, 64])
Target shape: torch.Size([32, 15])
Loss: 3.9698233604431152
Output shape: torch.Size([32, 15, 64])
Target shape: torch.Size([32, 15])
Loss: 3.921349287033081
Output shape: torch.Size([32, 15, 64])
Target shape: torch.Size([32, 15])
Loss: 3.845768690109253
Output shape: torch.Size([32, 15, 64])
Target shape: torch.Size([32, 15])
Loss: 3.796046495437622
Output shape: torc

In [27]:
model.eval()
with torch.no_grad():
    for misspelled, correct in test_loader:
        outputs = model(misspelled)
        predictions = torch.argmax(outputs, dim=-1)
        print("Misspelled:", "".join([train_dataset.idx2char[idx.item()] for idx in misspelled[0] if idx.item() != 0]))
        print("Predicted:", "".join([train_dataset.idx2char[idx.item()] for idx in predictions[0] if idx.item() != 0]))
        print("Correct:", "".join([train_dataset.idx2char[idx.item()] for idx in correct[0] if idx.item() != 0]))
        break


Misspelled: PddoT
Predicted: 
Correct: Pddg0m


In [28]:
model.eval()
total_correct = 0
total_count = 0

with torch.no_grad():
    for misspelled, correct in test_loader:
        outputs = model(misspelled)
        predictions = torch.argmax(outputs, dim=-1)
        
        # Flatten the tensors for comparison
        correct_flat = correct.view(-1)
        predictions_flat = predictions.view(-1)
        
        # Count non-padding tokens
        non_padding = correct_flat != 0
        
        # Calculate the number of correct predictions
        total_correct += (predictions_flat[non_padding] == correct_flat[non_padding]).sum().item()
        total_count += non_padding.sum().item()

# Calculate accuracy
accuracy = total_correct / total_count
print("Accuracy:", accuracy)

Accuracy: 0.0


In [29]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Convert words to token IDs
train_df["tokenized_misspelled"] = train_df["Misspelled_Word"].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
train_df["tokenized_correct"] = train_df["Correct_Word"].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))


In [31]:
# Load pre-trained BERT
bert_model = BertForMaskedLM.from_pretrained("bert-base-uncased")
bert_model.train()

optimizer = optim.Adam(bert_model.parameters(), lr=5e-5)

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize and pad sequences
def tokenize_and_pad(texts, tokenizer, max_length):
    tokenized_texts = [tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=max_length) for text in texts]
    padded_texts = pad_sequence([torch.tensor(text) for text in tokenized_texts], batch_first=True, padding_value=tokenizer.pad_token_id)
    return padded_texts

# Prepare inputs for training
max_length = 64 # Reduced max_length to a more typical value for BERT
input_ids = tokenize_and_pad(train_df["Misspelled_Word"].tolist(), tokenizer, max_length=max_length)
labels = tokenize_and_pad(train_df["Correct_Word"].tolist(), tokenizer, max_length=max_length)

# Ensure both input_ids and labels have the same shape
# by truncating to the minimum length if necessary
min_length = min(input_ids.shape[1], labels.shape[1])
input_ids = input_ids[:, :min_length]
labels = labels[:, :min_length]

# Training loop
for epoch in range(5):
    optimizer.zero_grad()
    outputs = bert_model(input_ids, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


: 